### Verifice della validità dei risulati ottenuto dalle diverse implementazioni

In [88]:
import csv
from sklearn.metrics import mean_squared_error

In [89]:
ouptut_SPQRK_SQL="../SPARK/out_Spark_SQL/output_SPARK_SQL.csv/output.csv"
output_SPARK_CORE="../SPARK/output/output.txt"
ouput_HIVE="../HIVE//output/output.csv"
output_MR="../MR/output2/output.txt"



In [90]:
#creazione del dizionario ordinato per userId
def convert_dic_user_score_text(data):
    user2score={}
    for i in data:
        i=i.strip()
        values=i.split('\t')
        if len(values)>1:
            
            userId=values[0]
            score_string=values[1]
            try:
                score=float(score_string)
            except ValueError:
                score=-1

            user2score[userId]=score
        
    
    user2score = dict(sorted(user2score.items()))          
    return user2score        


In [91]:
#creazione del dizionario ordinato per userId
def convert_dic_user_score_csv(data):
    
    user2score={}
    for values in data:
        
        if len(values)>1:
            userId=values[0]
            score_string=values[1]
            try:
                score=float(score_string)
            except ValueError:
                score=-1

            user2score[userId]=score
    
    user2score = dict(sorted(user2score.items()))          
    return user2score        


In [92]:
#creazione dei dizionari: SPARK-CORE e MR usano formato .txt, hive e sparksql formato csv (hanno headers)

#TXT
#MR
with open(output_MR,'r') as file:
    lines=file.readlines()
    
    MR_map=convert_dic_user_score_text(lines)
    print("Dim MR:",len(MR_map))

#SPARK_CORE
with open(output_SPARK_CORE,'r') as file:
    lines=file.readlines()
    SPARK_CORE_map=convert_dic_user_score_text(lines)
    print("DIM CORE:",len(SPARK_CORE_map))
#CSV
#SPARK_SQL
with open(ouptut_SPQRK_SQL,'r') as file:
    reader=csv.reader(file)
    next(reader)

    SPARK_SQL_map=convert_dic_user_score_csv(reader)
    print("DIM SQL:",len(SPARK_SQL_map))
#HIVE
with open(ouput_HIVE,'r') as file:
    reader=csv.reader(file)
    next(reader)
    HIVE_map=convert_dic_user_score_csv(reader)
    print("DIM CORE:",len(HIVE_map))


Dim MR: 256059
DIM CORE: 256059
DIM SQL: 256059
DIM CORE: 256059


In [93]:
print(list(HIVE_map.items())[:5])
print(list(SPARK_CORE_map.items())[:5])

print(list(MR_map.items())[:5])

print(list(SPARK_SQL_map.items())[:5])

[('#oc-R103C0QSV1DF5E', 0.5), ('#oc-R109MU5OBBZ59U', 0.0), ('#oc-R10LFEMQEW6QGZ', 0.0), ('#oc-R10LT57ZGIB140', 0.0), ('#oc-R10UA029WVWIUI', 0.0)]
[('#oc-R103C0QSV1DF5E', 0.5), ('#oc-R109MU5OBBZ59U', 0.0), ('#oc-R10LFEMQEW6QGZ', 0.0), ('#oc-R10LT57ZGIB140', 0.0), ('#oc-R10UA029WVWIUI', 0.0)]
[('#oc-R103C0QSV1DF5E', 0.5), ('#oc-R109MU5OBBZ59U', 0.0), ('#oc-R10LFEMQEW6QGZ', 0.0), ('#oc-R10LT57ZGIB140', 0.0), ('#oc-R10UA029WVWIUI', 0.0)]
[('#oc-R103C0QSV1DF5E', 0.5), ('#oc-R109MU5OBBZ59U', 0.0), ('#oc-R10LFEMQEW6QGZ', 0.0), ('#oc-R10LT57ZGIB140', 0.0), ('#oc-R10UA029WVWIUI', 0.0)]


In [94]:
def get_MSE(list1,list2):
    if len(list1) != len(list2):
        raise ValueError("Lists must have the same length")

    squared_errors = [(x - y) ** 2 for x, y in zip(list1, list2)]
    mse = sum(squared_errors) / len(list1)
    return mse

In [95]:
import itertools
list_of_results=[SPARK_CORE_map,SPARK_SQL_map,HIVE_map,MR_map]

mse_SPARK_CORE_SPARK_SQL=get_MSE(SPARK_CORE_map.values(),SPARK_SQL_map.values())
mse_SPARK_CORE_HIVE=get_MSE(SPARK_CORE_map.values(),HIVE_map.values())
mse_SPARK_CORE_MR=get_MSE(SPARK_CORE_map.values(),MR_map.values())
mse_SPARK_SQL_HIVE=get_MSE(SPARK_SQL_map.values(),HIVE_map.values())
mse_SPARK_SQL_MR=get_MSE(SPARK_SQL_map.values(),MR_map.values())
mse_HIVE_MR=get_MSE(HIVE_map.values(),MR_map.values())




In [109]:
print("SPARK-CORE,SPARK-sql",mse_SPARK_CORE_SPARK_SQL)
print("SPARK-CORE,Hive",mse_SPARK_CORE_HIVE)
print("SPARK-CORE,MR",mse_SPARK_CORE_MR)
print("SPARK-SQL,HIVE",mse_SPARK_SQL_HIVE)
print("SQL-MR",mse_SPARK_SQL_MR)

print("HIVE,MR",mse_HIVE_MR)

SPARK-CORE,SPARK-sql 1.1716049828535523e-05
SPARK-CORE,Hive 1.1143578915583794e-14
SPARK-CORE,MR 7.832044853233894e-15
SPARK-SQL,HIVE 1.1716049816704231e-05
SQL-MR 1.1716049822297047e-05
HIVE,MR 5.233875049702171e-15


In [108]:
def sample_differences(data1,data2):
    differences=[]
    sample_size=10
    conta=0
    for k in data1.keys():
        if conta==sample_size:
            break
        values_d1=data1[k]
        values_d2=data2[k]

        if values_d1!=values_d2:
            element=[k,values_d1,values_d2]
            differences.append(element)
            conta+=1
    return differences
        

In [110]:
sample_differences_MR_HIVE=sample_differences(SPARK_SQL_map,MR_map)
sample_differences_MR_HIVE

[['#oc-R115TNMSPFT9I7', 0.6666666666666666, 0.666667],
 ['#oc-R119LM8D59ZW8Y', 0.45454545454545453, 0.454545],
 ['#oc-R11D9LKDAN5NQJ', 0.6666666666666666, 0.666667],
 ['#oc-R13X3YIJ6GLT0C', 0.5714285714285714, 0.571429],
 ['#oc-R16HPROGAO1HS6', 0.3333333333333333, 0.333333],
 ['#oc-R1791MZMDMM68R', 0.5294117647058824, 0.529412],
 ['#oc-R17KN3F3UGQSU0', 0.3333333333333333, 0.333333],
 ['#oc-R18WCIOVC1O7U3', 0.3333333333333333, 0.333333],
 ['#oc-R18XETRRZHQVPE', 0.6666666666666666, 0.666667],
 ['#oc-R192YGWMM86YIU', 0.5555555555555556, 0.555556]]

In [111]:
sample_differences_MR_SPARK_CORE=sample_differences(SPARK_SQL_map,HIVE_map)
sample_differences_MR_SPARK_CORE

[['#oc-R115TNMSPFT9I7', 0.6666666666666666, 0.6666667],
 ['#oc-R119LM8D59ZW8Y', 0.45454545454545453, 0.45454547],
 ['#oc-R11D9LKDAN5NQJ', 0.6666666666666666, 0.6666667],
 ['#oc-R13X3YIJ6GLT0C', 0.5714285714285714, 0.5714286],
 ['#oc-R16HPROGAO1HS6', 0.3333333333333333, 0.33333334],
 ['#oc-R1791MZMDMM68R', 0.5294117647058824, 0.5294118],
 ['#oc-R17KN3F3UGQSU0', 0.3333333333333333, 0.33333334],
 ['#oc-R18WCIOVC1O7U3', 0.3333333333333333, 0.33333334],
 ['#oc-R18XETRRZHQVPE', 0.6666666666666666, 0.6666667],
 ['#oc-R192YGWMM86YIU', 0.5555555555555556, 0.5555556]]

# Validità locale dei risulati

In [100]:
#Sample UserId
samples_userId=list(MR_map.keys())[1090:1100]
samples_userId

['A109R6CX8Y1KG1',
 'A109RFVD8JUHJG',
 'A109UDZB70HFRF',
 'A109V1N9ZW77LT',
 'A109VBXN1OUQF3',
 'A109VTGM6ALZZ7',
 'A109VW0OXUQX45',
 'A109W8CR31K3ED',
 'A109WBWE4ZS0QT',
 'A109WE4H6C9O2X']

In [101]:
import pandas as pd

data=pd.read_csv("../../Dataset/Rev_Parsed.csv")

data=data[data['UserId'].isin(samples_userId)]


In [102]:

columns_Needed=["UserId","HelpfulnessNumerator","HelpfulnessDenominator"]

data=data[columns_Needed]

In [103]:
def carate_data_test(ids,data):
    test_list=[]
    for i in ids:
        data_test=data[data['UserId']==i]
        test_list.append(data_test)
    return test_list

In [104]:
import numpy as np
def compute_ratio(df):
     
    avg_ratio = df.apply(lambda row: row['HelpfulnessNumerator'] / row['HelpfulnessDenominator'] if row['HelpfulnessDenominator'] != 0 else 0, axis=1).sum()
    avg_ratio=avg_ratio/len(df)
   

    # Print the average ratio
    return avg_ratio

def get_results(ids,test_data):
    userId2result={}
    for i in range(0,len(test_data)):
        current_user=ids[i]
        current_dataset=test_data[i]
        result=compute_ratio(current_dataset)
        userId2result[current_user]=result
    return userId2result


In [105]:
#trovo gli opuptu
test_data=carate_data_test(samples_userId,data)


results=get_results(samples_userId,test_data)




In [106]:
print(results)

{'A109R6CX8Y1KG1': 1.0, 'A109RFVD8JUHJG': 1.0, 'A109UDZB70HFRF': 0.5, 'A109V1N9ZW77LT': 0.0, 'A109VBXN1OUQF3': 0.0, 'A109VTGM6ALZZ7': 0.0, 'A109VW0OXUQX45': 0.0, 'A109W8CR31K3ED': 0.5, 'A109WBWE4ZS0QT': 0.6666666666666666, 'A109WE4H6C9O2X': 1.0}


In [107]:
#check dei risulatati
for k in results.keys():
    current_user=k
    true_value=results[k]
    MR_value=MR_map[k]
    SPARK_CORE_value=SPARK_CORE_map[k]

    SPARK_SQL_value=SPARK_SQL_map[k]

    HIVE_value=HIVE_map[k]

    if true_value!=MR_value:
        print("ERRORE MR",MR_value)
    
    if true_value!=SPARK_CORE_value:
        print("ERRORE SC",current_user,true_value,SPARK_CORE_value)

    if true_value!=SPARK_SQL_value:
        print("ERRORE SSQL",current_user,true_value,SPARK_SQL_value)

    if true_value!=HIVE_value:
        print("ERRORE MR",current_user,true_value,HIVE_value)

ERRORE SC A109WBWE4ZS0QT 0.6666666666666666 0.666667
ERRORE MR A109WBWE4ZS0QT 0.6666666666666666 0.6666667


In [115]:
#¢ompute MSE for test data
MR_test_set={key: value for key, value in MR_map.items() if key in samples_userId}

SQL_test_set={key: value for key, value in SPARK_SQL_map.items() if key in samples_userId}

HIVE_test_set={key: value for key, value in HIVE_map.items() if key in samples_userId}

CORE_test_set={key: value for key, value in SPARK_CORE_map.items() if key in samples_userId}


10
10


In [116]:

mse_SPARK_SQL_test=get_MSE(CORE_test_set.values(),results.values())
mse_HIVE_test=get_MSE(results.values(),HIVE_test_set.values())
mse_SPARK_CORE_test=get_MSE(CORE_test_set.values(),results.values())

mse_MR_test=get_MSE(MR_test_set.values(),results.values())
print("SPARK-sql",mse_SPARK_SQL_test)
print("SPARK-CORE",mse_SPARK_CORE_test)
print("MR",mse_MR_test)
print("HIVE",mse_HIVE_test)


SPARK-sql 1.1111111114217288e-14
SPARK-CORE 1.1111111114217288e-14
MR 0.0
HIVE 1.1111111099414314e-16
